# 한글 오토마타
---
- QWERTY 자판 기준
- 강의노트 **한글처리.pdf(슬라이드 18-19)** 참조   
---
All rights reserved, 2021-2023 By **Youn-Sik Hong**. 수업 목적으로만 활용 가능.

### get_index(): 초성, 중성, 종성 테이블에서 해당 자모의 인덱스를 찾음 

In [1]:
def get_index (ch_set, ch):
    lim = len(ch_set)
    for i in range(lim):
        if ch == ch_set[i]:
            return i
    return -1 # not found

### is_jaum(): 한글 자음인지 조사

In [2]:
def is_jaum(ch):
    if 'ㅏ' <= ch <= 'ㅣ':
        return False
    return True 

초성, 중성 및 종성 문자열

In [3]:
uni_choSung = "ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ"
uni_joongSung ="ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ"
uni_jongSung = " ㄱㄲㄳㄴㄵㄶㄷㄹㄺㄻㄼㄽㄾㄿㅀㅁㅂㅄㅅㅆㅇㅈㅊㅋㅌㅍㅎ"

### make_han_syllable(): 유니코드 한글 음절(syllable) 생성

In [4]:
def make_han_syllable(cho, joong, jong):
    i = j = k = 0
    i = get_index(uni_choSung, cho) 
    j = get_index(uni_joongSung, joong) 
    k = get_index(uni_jongSung, jong)
    return chr(i*21*28 + j*28 + k + 44032)

### combine_jj() : 2개의 자음을 결합한 겹자음 생성 

In [5]:
def combine_jj(ja1, ja2):
    null_char = ' '
        
    if ja1 == 'ㄱ' :
        if ja2 == 'ㄱ': return 'ㄲ'
        if ja2 == 'ㅅ': return 'ㄳ'
        return null_char
        
    elif ja1 == 'ㄴ' :
        if ja2 == 'ㅈ': return 'ㄵ'
        if ja2 == 'ㅎ': return 'ㄶ'
        return null_char

    elif ja1 == 'ㄹ':
        if ja2 == 'ㄱ': return 'ㄺ'
        if ja2 == 'ㅁ': return 'ㄻ'
        if ja2 == 'ㅂ': return 'ㄼ'
        if ja2 == 'ㅅ': return 'ㄽ'
        if ja2 == 'ㅌ': return 'ㄾ'
        if ja2 == 'ㅍ': return 'ㄿ'
        if ja2 == 'ㅎ': return 'ㅀ'
        return null_char

    elif ja1 == 'ㅂ':
        if ja2 == 'ㅅ': return 'ㅄ'
        return null_char

    elif ja1 == 'ㅅ':
        if ja2 == 'ㅅ': return 'ㅆ'
        return null_char

    return null_char

### combine_mm() : 2개의 모음을 결합한 복모음 생성 

In [6]:
def combine_mm(self, mo1, mo2) :
    null_char = ' '
        
    if mo1 == 'ㅗ' :
        if mo2 == 'ㅏ': return 'ㅘ'
        if mo2 == 'ㅐ': return 'ㅙ'
        if mo2 == 'ㅣ': return 'ㅚ'
        return null_char

    if mo1 == 'ㅜ':
        if mo2 == 'ㅓ': return 'ㅝ'
        if mo2 == 'ㅔ': return 'ㅞ'
        if mo2 == 'ㅣ': return 'ㅟ'
        return null_char

    if mo1 == 'ㅡ':
        if mo2 == 'ㅣ': return 'ㅢ'
        return null_char

    return null_char

## HangleAutomata: 한글 오토마타 
- 코드 안에 주석을 참고하세요
- 강의노트 **한글처리.pdf(슬라이드 18-19)** 참조 

In [7]:
class HangleAutomata:
    def __init__(self, in_str):
        self.in_str = in_str
        self.out_str = []
        self.prev_ch = ' '  
        self.chosung = ' '
        self.joosung = ' '
        self.jongsung = ' '
        self.han_syllable = ' ' # 현재 받침이 다음 음절의 초성으로 분리되는 경우를 위해
                                # 이전에 완성된 한글 음절을 저장
        self.state = 0          # 상태 변수. 초기 상태

    def process(self):
        lim = len(self.in_str)
        #print(self.state)
        for i in range(lim):
            ch = self.in_str[i]
            self.stateTransition(ch)
            #print(self.state)
            
        self.out_str.append(self.han_syllable)
        return self.out_str
            
        
    def stateTransition(self, ch) :
        if self.state == 0: # 초기 상태
            if is_jaum(ch) == True: # 첫 번째 문자가 자음(초성) : 다음 상태 1
                self.prev_ch = ch
                self.state = 1
                
            else : # 첫 번째 문자가 모음 : 상태 천이 없이 시작 상태에 머뭄. 
                self.out_str.append(ch) # 입력한 모음을 그대로 출력.
                self.prev_ch = ' '
                self.state = 0

        elif self.state == 1: # 상태 1
            if is_jaum(ch) == True: # 자음을 또 입력. 초성은 겹자음만 허용(ㄲ, ㅆ, ...).                                                        
                self.out_str.append(prev_ch) # 이전 입력한 자음은 그대로 출력.
                self.prev_ch = ch   # 새로 입력된 자음을 저장.  
                self.state = 1      # 한개의 자음이 입력된 상태 1에 그대로 머뭄.
                
            else: # 모음 입력. [초성+중성] 꼴 구성. 상태 2로 이동.
                self.chosung = self.prev_ch 
                self.joongsung = ch 
                self.han_syllable = make_han_syllable(self.chosung, self.joongsung, ' ')
                self.prev_ch = ch
                self.state = 2

        elif self.state == 2: # 상태 2
            if is_jaum(ch) == True: # 자음 입력. 초성+중성+종성 꼴 구성. 상태 4로 이동.
                self.jongsung = ch 
                self.han_syllable = make_han_syllable(self.chosung, self.joongsung, self.jongsung)                
                self.prev_ch = ch
                self.state = 4
                
            else: # 모음 입력. 중성으로 겹모음이 입력됨. 상태 3으로 이동.
                # 2개의 모음을 결합해 한개의 겹모음 생성. 
                self.joongsung = combine_mm(self.prev_ch, ch) 
                self.han_syllable = make_han_syllable(self.chosung, self.joongsung, ' ')
                self.prev_ch = ch
                self.state = 3

        elif self.state == 3: # 상태 3
            if is_jaum(ch) == True: # 종성 자음. 초성+중성+종성 꼴 구성. 상태 5로 이동.
                self.jongsung = ch 
                self.han_syllable = make_han_syllable(self.chosung, self.joongsung, self.jongsung)
                self.prev_ch = ch
                self.state = 5
                
            else: 
                self.out_str.append(self.han_syllable)
                # unexpected char. 한글 음절을 구성할 수 없음. 
                self.out_str.append(ch)
                # 초기 상태로 되돌아감.
                self.state = 0

        elif self.state == 4: # 상태 4
            if is_jaum(ch) == True: 
                # 종성 자음이 또 입력되었지만, 구성 가능한 겹받침 조합이 아님. 
                # 현재 입력된 자음은 다음 음절의 초성으로 사용. 상태 1로 이동.
                if combine_jj(self.prev_ch, ch) == ' ':
                    self.out_str.append(self.han_syllable)
                    self.prev_ch = ch
                    self.state = 1
                    
                else:
                    # 종성 자음이 또 입력되었으며, 겹받침 중 하나. 상태 5로 이동.
                    self.jongsung = combine_jj(self.prev_ch, ch) 
                    self.han_syllable = make_han_syllable(self.chosung, self.joongsung, self.jongsung)
                    self.prev_ch = ch
                    self.state = 5

            else:
                # 모음 입력. 기존 음절의 받침으로 사용된 자음은 
                # 다음 음절의 초성으로 분리.
                self.han_syllable = make_han_syllable(self.chosung, self.joongsung, ' ')
                self.out_str.append(self.han_syllable)
                
                self.chosung = self.prev_ch 
                self.joongsung = ch 
                self.han_syllable = make_han_syllable(self.chosung, self.joongsung, ' ')
                self.prev_ch = ch
                # 다음 상태는 자음+모음 꼴이 구성된 상태 2.
                self.state = 2

        elif self.state == 5: # 상태 5
            if is_jaum(ch) == True: 
                self.out_str.append(self.han_syllable)
                 # 자음 입력. 새로운 한글 음절의 초성임. 상태 1로 이동. 
                self.prev_ch = ch
                self.state = 1
                
            else:
                # 모음 입력. 기존 음절의 받침으로 사용된 자음은 
                # 다음 음절의 초성으로 분리.
                self.out_str.append(self.han_syllable)
                self.chosung = self.prev_ch 
                self.joongsung = ch 
                self.han_syllable = make_han_syllable(self.chosung, self.joongsung, ' ')
                
                # 다음 상태는 자음+모음 꼴이 구성된 상태 2.
                self.prev_ch = ch
                self.state = 2

        elif self.state == 6: # 상태 6
            if is_jaum(ch) == True: 
                self.out_str.append(self.han_syllable)
                 # 자음 입력. 새로운 한글 음절의 초성임. 상태 1로 이동. 
                self.prev_ch = ch
                self.state = 1   
                
            else: 
                # 모음 입력. 기존 음절의 받침으로 사용된 자음은 
                # 다음 음절의 초성으로 분리.
                # 다음 상태는 자음+모음 꼴이 구성된 상태 2.
                self.out_str.append(self.han_syllable)
                self.chosung = self.prev_ch 
                self.joongsung = ch 
                self.han_syllable = make_han_syllable(self.chosung, self.joongsung, ' ')
                
                self.prev_ch = ch
                self.state = 2

- 테스트를 하려면, 아래 예처럼 한글을 자모 분리해서 문자열을 만들어 입력해야 합니다.
- 실제 키보드롤 입력할 때 입력 방식과 같지만, 
    - 최대한 간단히 구현하기 위해 
        - 한글 자모만 입력했다고 가정하였으며,
        - 한글 외에 영문자를 한글로 바꾸는 등의 처리는 하지 않았습니다.

In [8]:
h = HangleAutomata('ㅎㅏㄴㅡㄹ')
h.process()

['하', '늘']

In [9]:
h = HangleAutomata('ㅇㅓㅈㅔㅇㅏㅊㅣㅁ')
h.process()

['어', '제', '아', '침']

In [10]:
h = HangleAutomata('ㄱㅏㅂㅅㅇㅣㅂㅣㅆㅏㄷㅏ')
h.process()

['값', '이', '비', '싸', '다']

In [11]:
h = HangleAutomata('ㄱㅣㄹㅇㅡㄹㄸㅏㄹㅏㄱㅓㄷㄷㅏ')
h.process()

['길', '을', '따', '라', '걷', '다']